# Configuração do SageMaker Estimator

## 1 - Configuração Inicial:

In [3]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.inputs import TrainingInput

# Obter o role e sessão do SageMaker
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

# Definir o bucket e os caminhos no S3
bucket = 'sagemaker-us-east-1-837612112902'
prefix = 'telco_churn'
train_data_uri = f's3://{bucket}/{prefix}/dataset/'
output_path = f's3://{bucket}/{prefix}/output/'

## 2 - Configuração do Estimador:

In [ ]:
# Configurar o Estimador para o treinamento com XGBoost
xgb_estimator = SKLearn(
    entry_point="train.py",  # O script de treinamento
    role=role,               # Role do SageMaker
    instance_type="ml.m5.large",  # Tipo de instância
    instance_count=1,        # Número de instâncias
    framework_version="0.23-1",  # Versão do Scikit-learn
    py_version="py3",
    output_path=output_path,  # Diretório S3 para armazenar o modelo,
    dependencies=["requirements.txt"],  # Inclui as dependências
    hyperparameters={         # Hiperparâmetros para o XGBoost
        'n_estimators': 100,
        'max_depth': 5,
        'learning_rate': 0.1
    }
)

## 3 - Iniciar Treinamento:

In [ ]:
# Usar TrainingInput para passar os dados corretamente
train_input = TrainingInput(s3_data=train_data_uri, content_type='csv')

# Iniciar o treinamento com os dados do S3
xgb_estimator.fit({'train': train_input})

## 4 - Registro do Modelo:

In [ ]:
#xgb_estimator.model_data

In [18]:
model_artifact = 's3://sagemaker-us-east-1-837612112902/telco_churn/output/sagemaker-scikit-learn-2024-10-15-22-48-20-490/output/model.tar.gz'

In [11]:
from sagemaker.sklearn.model import SKLearnModel

# Definir o modelo usando o artefato treinado
model = SKLearnModel(
    model_data=model_artifact,  # Local onde o modelo foi salvo no S3 após o treinamento
    role=role,
    entry_point="predict.py",  # Mesmo script de treinamento usado para inferência
    framework_version="0.23-1",  # Versão do Scikit-learn utilizada
    py_version="py3"
)


In [6]:
# Configurar o transformador para predições em batch
transformer = model.transformer(
    instance_count=1,  # Número de instâncias para a inferência
    instance_type="ml.m5.large",  # Tipo de instância
    output_path="s3://sagemaker-us-east-1-837612112902/telco_churn/predictions/",  # Caminho para salvar as predições
)

In [ ]:
model_artifact = 's3://sagemaker-us-east-1-837612112902/telco_churn/output/sagemaker-scikit-learn-2024-10-15-22-48-20-490/output/model.tar.gz'

In [ ]:
from sagemaker.sklearn.model import SKLearnModel

# Definir o modelo usando o artefato treinado
model = SKLearnModel(
    model_data=model_artifact,  # Local onde o modelo foi salvo no S3 após o treinamento
    role=role,
    entry_point="predict.py",  # Script para realizar a inferência
    framework_version="0.23-1",  # Versão do Scikit-learn utilizada
    py_version="py3"
)

## 5 - Inferência em Batch:

In [ ]:
# Configurar o transformador para predições em batch
transformer = model.transformer(
    instance_count=1,  # Número de instâncias para a inferência
    instance_type="ml.m5.large",  # Tipo de instância
    output_path="s3://sagemaker-us-east-1-837612112902/telco_churn/predictions/",  # Caminho para salvar as predições
)


## 6 - Implantação de Endpoint:

In [19]:
from sagemaker.sklearn.model import SKLearnModel

# Definir o modelo usando o artefato treinado
model = SKLearnModel(
    model_data=model_artifact,
    role=role,
    entry_point='inference.py',
    framework_version='0.23-1',
    py_version='py3',
    dependencies=['requirements.txt']
)

In [20]:
# Implantar o modelo como um endpoint
predictor = model.deploy(
    initial_instance_count=1,  # Número de instâncias
    instance_type='ml.m5.large'  # Tipo de instância
)

------!